<a href="https://colab.research.google.com/github/yarusx/cat_vs_dogo/blob/main/cats_dogs_breed_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pathlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/images2.zip

In [ ]:
data_dir = pathlib.Path('/content/images')

In [ ]:
image_count = len(list(data_dir.glob('*/*/*.jpg')))
print(image_count)

7390


In [ ]:
Cats = list(data_dir.glob('Cat/*/*'))
PIL.Image.open(str(Cats[0]))

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels='inferred',
  label_mode='int',
  class_names = ['Cat','Dog'],
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

Found 7390 files belonging to 2 classes.
Using 5912 files for training.


In [ ]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  labels='inferred',
  label_mode='int',
  class_names = ['Cat','Dog'],
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=IMG_SIZE,
  batch_size=BATCH_SIZE)

Found 7390 files belonging to 2 classes.
Using 1478 files for validation.


In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

tf.Tensor([0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 0 1], shape=(32,), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 0 1], shape=(32,), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 0 1], shape=(32,), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 0 1], shape=(32,), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 0 1], shape=(32,), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 0 1], shape=(32,), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([0 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0

<Figure size 720x720 with 0 Axes>

In [ ]:
print(class_names)

['Cat', 'Dog']
